<a href="https://colab.research.google.com/github/johnlocke333/xray_image_classification/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data Preparation**


In [2]:
!pip install scikeras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 58.0 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [3]:
!pip install scikit-learn==1.2.2

  Using cached scikit_learn-1.2.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
Using cached scikit_learn-1.2.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (9.6 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlxtend 0.23.3 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatible.
scikeras 0.13.0 requires scikit-learn>=1.4.2, but you have scikit-learn 1.2.2 which is incompatible.


In [4]:
from IPython import get_ipython
from IPython.display import display

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import array_to_img, img_to_array, load_img
import numpy as np
import os

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!unzip -uq "/content/drive/My Drive/xray.zip" -d "/content/drive/My Drive/xray"

In [7]:
# Directory path
train_data_dir = '/content/drive/MyDrive/xray/chest_xray/chest_xray/train'
test_data_dir = '/content/drive/MyDrive/xray/chest_xray/chest_xray/test'
valid_data_dir = '/content/drive/MyDrive/xray/chest_xray/chest_xray/val'


In [8]:
def get_reshape_images(dir, bs):
# Get all the data in the directory data/validation (5216 images), and reshape them, color to grayscale because they are xray images
  generator = ImageDataGenerator().flow_from_directory(
          dir, target_size=(64, 64), batch_size=bs, color_mode='grayscale')

# Create the datasets
  images, labels = next(generator)
  return images, labels

In [9]:
train_images, train_labels = get_reshape_images(train_data_dir, 5216)

Found 5216 images belonging to 2 classes.


In [10]:
test_images, test_labels = get_reshape_images(test_data_dir, 624)

Found 624 images belonging to 2 classes.


In [11]:
valid_images, valid_labels = get_reshape_images(valid_data_dir, 16)

Found 16 images belonging to 2 classes.


In [12]:
 def reshape_input_image_label(images, m, labels):
  img_unrow = images.reshape(m, -1)
  labels_final = labels.T[[1]].T
  return img_unrow, labels_final

In [13]:
train_img_unrow, train_label_final = reshape_input_image_label(train_images, 5216, train_labels)

In [14]:
test_img_unrow, test_label_final = reshape_input_image_label(test_images, 624, test_labels)

In [15]:
valid_img_unrow, valid_label_final = reshape_input_image_label(valid_images, 16, valid_labels)

In [16]:
def standardize(img_unrow):
  img_final = img_unrow/255
  return img_final

In [17]:
train_img_final = standardize(train_img_unrow)

In [18]:
test_img_final= standardize(test_img_unrow)

In [19]:
valid_img_final = standardize(valid_img_unrow)

# **Model**

In [20]:
import tensorflow as tf
from tensorflow import keras
from keras import models
from keras import layers
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [109]:
def build_baseline(optimizer='SGD', activation='relu',neurons=1):
  baseline_model = models.Sequential()
  baseline_model.add(layers.Dense(neurons, activation=activation, input_shape=(4096,)))
  baseline_model.add(layers.Dense(1, activation='sigmoid'))
  baseline_model.compile(optimizer=optimizer,
                        loss='binary_crossentropy',
                        metrics=['recall'])
  return baseline_model


In [110]:
baseline_model = build_baseline()
baseline_model.fit(train_img_final,
                    train_label_final,
                    epochs=6,
                    batch_size=3,
                    validation_data=(test_img_final, test_label_final))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/6
1739/1739 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 0.6124 - recall: 0.9908 - val_loss: 0.6934 - val_recall: 1.0000
Epoch 2/6
1739/1739 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.5735 - recall: 1.0000 - val_loss: 0.6938 - val_recall: 1.0000
Epoch 3/6
1739/1739 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - loss: 0.5809 - recall: 1.0000 - val_loss: 0.6993 - val_recall: 1.0000
Epoch 4/6
1739/1739 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.5717 - recall: 1.0000 - val_loss: 0.6984 - val_recall: 1.0000
Epoch 5/6
1739/1739 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.5641 - recall: 1.0000 - val_loss: 0.6914 - val_recall: 1.0000
Epoch 6/6
1739/1739 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - loss: 0.5672 - recall: 1.0000 - val_loss: 0.6939 - val_recall: 1.0000


In [111]:
tuning_model=KerasClassifier(build_fn=build_baseline)

In [112]:
param_grid = {
    "model__neurons": [1,2,5],
    "model__optimizer": ['SGD', 'adam', 'RMSProp'],
    "model__activation" : ['relu', 'tanh', 'sigmoid'],
    "epochs": [6,8,10],
    "batch_size" : [3,6,10,20]
}

In [113]:
gs=GridSearchCV(estimator=tuning_model, param_grid=param_grid, cv=3)
# now fit the dataset to the GridSearchCV object.
gs = gs.fit(test_img_final, test_label_final)

Epoch 1/6


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


139/139 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.7027 - recall: 0.9066
Epoch 2/6
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6721 - recall: 1.0000
Epoch 3/6
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6681 - recall: 1.0000
Epoch 4/6
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6633 - recall: 1.0000
Epoch 5/6
139/139 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.6595 - recall: 1.0000
Epoch 6/6
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6664 - recall: 1.0000
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch 1/6


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


139/139 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.6823 - recall: 1.0000
Epoch 2/6
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6459 - recall: 1.0000
Epoch 3/6
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.6017 - recall: 1.0000
Epoch 4/6
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5415 - recall: 0.9665
Epoch 5/6
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5113 - recall: 0.8993
Epoch 6/6
139/139 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.5260 - recall: 0.8455
70/70 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Epoch 1/6


...

21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.6867 - recall: 0.8466
Epoch 2/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6052 - recall: 1.0000
Epoch 3/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5884 - recall: 0.9055
Epoch 4/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5158 - recall: 0.8904
Epoch 5/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.4873 - recall: 0.9418
Epoch 6/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5024 - recall: 0.8692
Epoch 7/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.4427 - recall: 0.9304
Epoch 8/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.4783 - recall: 0.8666
Epoch 9/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4543 - recall: 0.9108
Epoch 10/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.4479 - recall: 0.9203
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.6858 - recall: 1.0000
Epoch 2/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.6353 - recall: 1.0000
Epoch 3/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5893 - recall: 1.0000
Epoch 4/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5734 - recall: 1.0000
Epoch 5/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5568 - recall: 1.0000
Epoch 6/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5350 - recall: 1.0000
Epoch 7/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5283 - recall: 0.9875
Epoch 8/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.5194 - recall: 1.0000
Epoch 9/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.4739 - recall: 0.9916
Epoch 10/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.4896 - recall: 0.9956
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.6782 - recall: 0.6246
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5608 - recall: 0.9547
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.5032 - recall: 0.9313
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.4505 - recall: 0.9518
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4687 - recall: 0.9130
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4182 - recall: 0.9499
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.4233 - recall: 0.9072
Epoch 8/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3949 - recall: 0.9346
Epoch 9/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3829 - recall: 0.9545
Epoch 10/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3595 - recall: 0.9538


In [114]:
best_params=gs.best_params_
recall=gs.best_score_

In [115]:
best_params

{'batch_size': 10,
 'epochs': 10,
 'model__activation': 'sigmoid',
 'model__neurons': 5,
 'model__optimizer': 'adam'}

In [116]:
recall

0.875

In [128]:
tuned_model = models.Sequential()
tuned_model.add(layers.Dense(5, activation='sigmoid', input_shape=(4096,)))
tuned_model.add(layers.Dense(1, activation='sigmoid'))
tuned_model.compile(optimizer='adam',
                      loss='binary_crossentropy',
                      metrics=['recall'])
tuned_model.fit(train_img_final,
                    train_label_final,
                    epochs=10,
                    batch_size=10,
                    validation_data=(test_img_final, test_label_final))

Epoch 1/10
522/522 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.6015 - recall: 1.0000 - val_loss: 0.7062 - val_recall: 1.0000
Epoch 2/10
522/522 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5801 - recall: 1.0000 - val_loss: 0.6984 - val_recall: 1.0000
Epoch 3/10
522/522 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5758 - recall: 1.0000 - val_loss: 0.6985 - val_recall: 1.0000
Epoch 4/10
522/522 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5852 - recall: 1.0000 - val_loss: 0.6981 - val_recall: 1.0000
Epoch 5/10
522/522 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5703 - recall: 1.0000 - val_loss: 0.6940 - val_recall: 1.0000
Epoch 6/10
522/522 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.5661 - recall: 1.0000 - val_loss: 0.6947 - val_recall: 1.0000
Epoch 7/10
522/522 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.5730 - recall: 1.0000 - val_loss: 0.6967 - val_recall: 1.0000
Epoch 8/10
522/522 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.5730 - recall: 1.0000 - val_loss: 0.6971 - val_recall: 1.0000
Epoch 9/

In [129]:
results_train = tuned_model.evaluate(train_img_final, train_label_final)
results_train

163/163 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5711 - recall: 1.0000


[0.5700041651725769, 1.0]

In [130]:
results_test = tuned_model.evaluate(test_img_final, test_label_final)
results_test

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7176 - recall: 1.0000


[0.6939619183540344, 1.0]